![../docs/images/banner.png](../docs/images/banner.png)


# [Contact Us](mailto:info@inverted.ai) for the server ip address.

In [ ]:
! pip install invertedai_client
from IPython.display import clear_output
clear_output()

In [ ]:
from invertedai_client.utils import Res, SensorSettings, Resolution, PyGameWindow, ClientSideBoundingBoxes
from invertedai_client.interface import IAIEnv, ServerTimeoutError
import numpy as np
import time
import cv2
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pygame
import argparse

In [ ]:
def_res = Res.SD
scale = 1
num_cams = 2
sensors_dict = {
        f'cam_{ind}':{
            'sensor_type': 'camera',
            'camera_type': 'rgb-camera',
            'resolution': def_res,
            'radius': 20,
            'occlusion': True,
            'fov': 90.0,
            } for ind in range(num_cams)}

In [ ]:
parser = argparse.ArgumentParser()
IAIEnv.add_config(parser)
config = parser.parse_args(['--client_id', 'ford'])
server_ip = input('Enter server IP:')
config.zmq_server_address = f"{server_ip}:5555"
world_parameters = dict(carlatown='Town04')
scenario_parameters = dict(camera_location_variation=SensorSettings.Location(x=1, z=1, y=1),
                           camera_rotation_variation=SensorSettings.Rotation(yaw=10, pitch=10, roll=10))
obs = None
while obs is None:
    env = IAIEnv(config)
    try:
        obs = env.set_scenario('worlddataset', world_parameters=world_parameters, scenario_parameters=scenario_parameters, sensors=sensors_dict)
    except ServerTimeoutError as e:
        print(f'Server timed out, retrying ...')
        env.close()
        continue

In [ ]:
pygame.init()

dim = (scale*def_res.width, scale*def_res.height)
width = np.sum([sensors_dict[sns]['resolution'].width*scale for sns in sensors_dict if sensors_dict[sns]['sensor_type']=='camera'])
height = np.max([sensors_dict[sns]['resolution'].height*scale for sns in sensors_dict if sensors_dict[sns]['sensor_type']=='camera'])
full_res = Resolution(width, height)
main_display = PyGameWindow(full_res)
##

In [ ]:
action = [0.0, 0.0]
obs, reward, done, info = env.step(action)
disp_img = np.concatenate(list(cv2.resize(obs['sensor_data'][name]['image'], dim, interpolation = cv2.INTER_AREA) for name in sensors_dict if 'image' in obs['sensor_data'][name].keys()), axis=1)
main_display.render(disp_img)
pygame.display.update()
rem_self = 0
howlong = 0
while not done:
    prev_action = obs['prev_action']
    if rem_self < howlong:
        action = info['expert_action']
        obs, reward, done, info = env.step(action)
        rem_self += 1
    else:
        cmd = input('Enter Steering:')
        if cmd == 'reset':
            obs = env.reset()
            jupyter_render(obs)
        elif cmd == 'init':
            obs = env.set_scenario('worlddataset', world_parameters=world_parameters, scenario_parameters=scenario_parameters, sensors=sensors_dict)
        elif cmd == 'end':
            print(env.end_simulation())
            break
        elif cmd == 'self':
            howlong = int(input('How many steps:'))
            rem_self = 0
        else:
            angle = float(cmd)
            acceleration = float(input('Enter Acceleration:'))
            action = (acceleration, angle)
            obs, reward, done, info = env.step(action)
    clear_output(wait=True)
    for attached_sensor in sensors_dict:
        name = 'boundingbox_'+attached_sensor
        if (name in obs['sensor_data']):
            bb2d = ClientSideBoundingBoxes.get_2d_bbox(obs['sensor_data'][name]['bounding_boxes'], 0, 0, sensors_dict[attached_sensor]['fov'], sensors_dict[attached_sensor]['resolution'], obs['compact_vector'][:3], obs['compact_vector'][3:6], coordinate_system='attached_sensor', occlusion=True)
            img = obs['sensor_data'][attached_sensor]['image']
            obs['sensor_data'][attached_sensor]['image'] = ClientSideBoundingBoxes.draw_bounding_boxes_on_array(img, bb2d, draw2d=False, occlusion=True)

   
    disp_img = np.concatenate(list(cv2.resize(obs['sensor_data'][name]['image'], dim, interpolation = cv2.INTER_AREA) for name in sensors_dict if 'image' in obs['sensor_data'][name].keys()), axis=1)
    main_display.render(disp_img)
    pygame.display.update()

print(f'Episode Done, Reward:{reward}')


In [ ]:
print(env.end_simulation())
env.close()